In [ ]:
# Scene Recognition and Segmentation with Deep Learning

## Outline

In this project, we will use *convolutional neural nets* to classify images into different scenes, and perform segmentation of images.

Basic learning objectives of this project:
1. Construct the fundamental pipeline for performing deep learning using PyTorch;
2. Understand the concepts behind different layers, optimizers.
3. Experiment with different models and observe the performance.

The starter code is mostly initialized to 'placeholder' just so that the starter
code does not crash when run unmodified and you can get a preview of how
results are presented.

## Compute Requirements

This classification part of the project is doable without a GPU, but a GPU makes the process much more faster and frustration free.

The segmentation part of the project needs a GPU to make the process faster and feasible. We have provided a separate notebook `proj5_code/proj5_colab.ipynb` for training with Colab for Segmentation.

You can try out Google Colab to run this notebook. These are the steps we follow:
1. Upload this notebook to google colab
2. Zip all the components in the project directory and upload it to the colab runtime
3. Unzip the uploaded zip using ```!unzip -qq <uploaded_file>.zip -d ./```

Remember to download all the files saved and changes to code you made on the colab.

Note that we will not be actively supporting issues with Google colab. Please take help from fellow students and use the internet to solve the issues.

## Dataset
You would have already stored the dataset in the ```data``` folder. It is the same data you have used in PS4. It has two subfolders: ```train``` and ```test```. Go through any of the folder and find there you will find the folders with scene names like *bedroom*, *forest*, *office*. These are the 15 scenes that we want our model to predict given an RGB image. You can look into folder for each scene to find multiple images. All this data is labelled data provided to you for training and testing your model.

**Let's start coding now!**

## Setup

In [ ]:
# uncomment for running on colab
# !unzip -qq proj5_colab.zip -d ./

In [ ]:
# flag to modify paths to run better on Colab; change it to true if you want to run on colab
use_colab_paths = False 

In [ ]:
!cd

## Setup

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder

from proj5_code.classification.runner import Trainer
from proj5_code.classification.optimizer import compute_quadratic_loss, get_optimizer, gradient_descent_step
from proj5_code.classification.simple_net import SimpleNet
from proj5_code.classification.simple_net_dropout import SimpleNetDropout
from proj5_code.classification.data_transforms import get_fundamental_transforms, get_data_augmentation_transforms
from proj5_code.classification.stats_helper import compute_mean_and_std
from proj5_code.classification.vis import visualize

%load_ext autoreload
%autoreload 2

In [ ]:
from proj5_code.proj5_unit_tests.test_base import verify
from proj5_code.proj5_unit_tests.classification.test_stats_helper import test_mean_and_variance
from proj5_code.proj5_unit_tests.classification.test_data_transforms import test_fundamental_transforms
from proj5_code.proj5_unit_tests.classification.test_dl_utils import test_predict_labels, test_compute_loss
from proj5_code.proj5_unit_tests.classification.test_simple_net import test_simple_net
from proj5_code.proj5_unit_tests.classification.test_simple_net_dropout import test_simple_net_dropout
from proj5_code.proj5_unit_tests.classification.test_checkpoints import test_simple_net_checkpoint
from proj5_code.proj5_unit_tests.classification.test_optimizer import test_compute_quadratic_loss, test_gradient_descent_step

In [ ]:
is_cuda = True
# torch.cuda.is_available() checks if there is a GPU on the system and it has CUDA drivers installed
# CUDA helps accelerate operations in Pytorch
is_cuda = is_cuda and torch.cuda.is_available() 

In [ ]:
data_base_path = '../data/' if not use_colab_paths else 'data/'
model_base_path = '../model_checkpoints/' if not use_colab_paths else 'model_checkpoints/'

In [ ]:
print(os.listdir(data_base_path))
print(os.listdir(model_base_path))

# TODO: check that these outputs are as per expectation. It will save a lot of time in debugging issues.

To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

## 1 Dataset
The dataset is in the ```data``` folder. It has two subfolders: ```train``` and ```test```. Go through any of the folder and find there you will find the folders with scene names like *bedroom*, *forest*, *office*. These are the 15 scenes that we want our model to predict given an RGB image. You can look into folder for each scene to find multiple images. All this data is labelled data provided to you for training and testing your model.

**Let's start coding now!**

One crucial aspect of deep learning is to perform data preprocessing. In DL, we usually *normalize* the dataset and perform some *transformations* on them. The transformations can either help the inputs be compatible with the model (say our model only works on 500x500 images and we need all input to be cropped/scaled to this size) or help in data-augmentation to improve performance (more on this later).


### 1.1 Compute mean and standard deviation of the dataset
In this project we are going to "zero-center" and "normalize" the dataset so that each entry has zero mean and the overall standard deviation is 1. To do this, we will compute the mean and variance of the actual dataset first. To perform actual normalization, we will use a transform from Pytorch. 

**TODO 1**:  fill in the `compute_mean_and_std()` in `proj5_code/classification/stats_helper.py` to compute the **mean** and **standard deviation** of both training and test set combined.

Debug tip: If you face an error from StandardScaler about attribute not found, please check that the paths are correct and your code actually finds some images.

In [ ]:
print("Testing your mean and std computation: ", verify(test_mean_and_variance))
dataset_mean, dataset_std = compute_mean_and_std(data_base_path)

In [ ]:
print('Dataset mean = {}, standard deviation = {}'.format(dataset_mean, dataset_std))

### 1.2 Data Loader

We'll have stored our input images in a specific format: each class name is a folder, with images belonging to that class inside the folder. This specific choice of storage scheme enables us to directly use the [`ImageFolder` ](https://pytorch.org/vision/stable/datasets.html#imagefolder) dataset from Pytorch.

We'll use the `ImageFolder` to make two loaders, for train and test split respectively.

In [ ]:
train_loader = ImageFolder(root=os.path.join(data_base_path, 'train'))
test_loader = ImageFolder(root=os.path.join(data_base_path, 'test'))

In [ ]:
print(f"Number of entries in training set: {len(train_loader)}")
print(f"Number of entries in test set: {len(test_loader)}")

Visualizing some examples from the training set

In [ ]:
idx = 100

image, class_label = train_loader.__getitem__(idx)
plt.figure()
plt.imshow(image)
plt.title(f'Image at index {idx}. Class Label {class_label}')
plt.show()

In [ ]:
idx = 300

image, class_label = train_loader.__getitem__(idx)
plt.figure()
plt.imshow(image)
plt.title(f'Image at index {idx}. Class Label {class_label}')
plt.show()

### 1.3 Data transforms
In this section, we will construct some fundamental transforms to process RGB images into torch tensors, which we can provide as input to our model.

1. Resize the input image to the desired shape
2. Convert to a single channel grayscale image (ImageFolder will read the image as a color image with 3 channels, even though the colors are all gray).
3. Convert it to a tensor
4. Normalize them based on the mean and standard deviation already computed in 1.1.

All these operations can be done using [torchvision's transforms](https://pytorch.org/vision/stable/transforms.html). You'll need to write one line for each operation.

**TODO 2:** For this part, complete the function `get_fundamental_transforms()` in `proj5_code/classification/data_transforms.py` 

In [ ]:
print("Testing your fundamental data transforms: ", verify(test_fundamental_transforms))

## 2 Model Architecture and Forward Pass

In [ ]:
inp_size = (64,64) # input size to use for this section

### 2.1 SimpleNet Model

The data is ready! Now we are preparing to move to the actual core of deep learning: the architecture. To get you started in this part, simply define a **2-layer** model in the `proj5_code/classification/simple_net.py`. Here by "2 layers" we mean **2 convolutional layers**, so you need to figure out the supporting utilities like ReLU, Max Pooling, and Fully Connected layers, and configure them with proper parameters (such as kernel size, padding, etc) such that the output from a layer is compatible as input to the next layer.

You may refer the image *simplenet.jpg* in the base folder for a sample network architecture (it's the architecture TAs used in their implementation and is sufficient to get you pass Part 1). In that image, the layers in use are written at the bottom, and the blocks show the shape of features after each layer.

Note that the input image will be shaped as (N, C, H, W) where N are the number of images in a batch, C is the number of color channels in the image, H is the height, and W is the width.

**TODO 3**: Do the following in ```proj5_code/classification/simple_net.py```:
- Initialize ```self.conv_layers```
- Initialize ```self.fc_layers```
- Write the forward function

Leave the ```self.loss_criterion = None``` for now.

In [ ]:
print("Testing your SimpleNet architecture: ", verify(test_simple_net))

### 2.2 Output prediction

Let's see what out model's forward function produces for a sample input, and how it relates to classification. Pytorch's convolution and FC layers are initialized with random weights. So we should not expect any useful output without any training.

We will use a data-point from a new `ImageFolder` object with the transformations we have defined.

In [ ]:
simple_model = SimpleNet()

train_loader = ImageFolder(root=os.path.join(data_base_path, 'train'), 
                           transform=get_fundamental_transforms(inp_size, dataset_mean, dataset_std))

In [ ]:
# get the 0th sample
sample_image, sample_label = next(iter(train_loader))

In [ ]:
print('Input image shape = ', sample_image.shape)

In [ ]:
#show the image
fig, axs = plt.subplots()
axs.imshow(sample_image.squeeze().numpy(), cmap='gray')
axs.axis('off')
axs.set_title('Sample image (Target label = {})'.format(sample_label))
plt.show()

In [ ]:
# run the image through the model
sample_model_output = simple_model(sample_image.unsqueeze(0)).detach()

In [ ]:
print(sample_model_output)

We have a 15-dimensional tensor as output, but how does it relate to classification?

We first convert the this tensor into a probability distribution over 15 classes by applying the [Softmax](https://en.wikipedia.org/wiki/Softmax_function) operation.

In [ ]:
sample_probability_values = torch.nn.functional.softmax(sample_model_output, dim=1)

In [ ]:
print(sample_probability_values)

In [ ]:
print("Sum of probability values: ", torch.sum(sample_probability_values))

The prediction of the model will be the index where the probability distribution is the maximum. Convince yourself that the argmax-operation on *sample_model_values* is the same as the argmax-operation on *sample_probability_values*. Hence for prediction, we can take the argmax on the model output directly without applying a softmax. Also, you can check that the sum of softmax values is 1 and hence they are valid probability values.

**TODO 4:** Complete the ```predict_labels()``` function in ```proj5_code/classification/dl_utils.py```.

## 3 Loss function

We have written a model which takes in a tensor for an image and produces a 15 dimensional output for it. We saw in the previous section on how the output relates to the prediction and probability distribution. But how do we quantify the performance of the model, and how do we use that quantification to form an objective function which we can minimize.

Ideally, we would want the probability function to have value 1 for the target *sample_label* and value 0 for the remaining class indices. To penalize the deviation between the desired probability distribution and the model-predicted distrtibution, we use the KL-divergence loss or the cross-entropy loss. Please refer to [this stackexchange post](https://datascience.stackexchange.com/questions/20296/cross-entropy-loss-explanation) for a good explanation and derivation.

Note: nn.CrossEntropyLoss() performs softmax itself, but nn.KLDivLoss() does not. Also, for KL-divergence loss, the ground truth has to be a distribution, not a numerical class label. Keep these in mind for the next TODO.

**TODO 5:** Assign a loss function to ```self.loss_criterion``` in ```proj5_code/classification/simple_net.py```. Note that we have not done a softmax operation in the model's forward function and choose the [appropriate loss function](https://pytorch.org/docs/stable/nn.html#loss-functions). In this TODO, you just have to assign the correct loss function in the model. 

**TODO 6:** Complete the ```compute_loss()``` function in ```proj5_code/classification/dl_utils.py``` to use the model's loss criterion and compute the loss.

In [ ]:
simple_model = SimpleNet()

In [ ]:
print(simple_model)

In [ ]:
print("Testing your model prediction: ", verify(test_predict_labels))
print("Testing your loss values: ", verify(test_compute_loss))

## 4 Optimizer

### 4.1 Manual gradient descent using Pytorch's autograd

Till now, we have defined the model, and designed a loss function which is a proxy for *good* classification. We now to optimize the weights of the network so that the loss function is minimized.

Pytorch is a very useful library for deep learning because a lot of tensor operations and functions support the flow of gradients. This feature is called [autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html). This functionality lets use use gradient based optimization techniques like gradient descent without writing a lot of code.

Let us first understand how we can access the gradients.

### Define an objective function (equivalent to the loss we defined in Section 3)
Suppose we have a simple objective function that looks like:
$$ L(w) =  w^2 - 10w + 25 $$

This is a convex problem, and we know that the loss $L$ is minimized for $w=5$, and we can obtain this in closed form.

But let us use gradient descent to obtain the solution in this case.

**TODO 7:** Complete `compute_quadratic_loss` in `proj5_code/classification/optimizer.py`

In [ ]:
print("Testing your quadratic loss function: ", verify(test_compute_quadratic_loss))

Let's compute the loss at w = 0.

In [ ]:
w = torch.tensor([0.0], requires_grad=True) # we mark this tensor to require gradient so that the computation graph
# can account for it

loss = compute_quadratic_loss(w)

print('w={:.4f}\tLoss={:.4f}'.format(w.detach().numpy().item(), loss.detach().numpy().item()))

Now we can do a backward pass of the gradients to get the gradient of loss w.r.t w. Now we need to calculate the gradients with regard to the weights and biases using backprop. It will be very painful if we do it manually, but thankfully, in PyTorch we can have everything covered with autograd, which only needs a simple call of **.backward()** on our loss.

In [ ]:
# perform a backward pass on loss (we need to retain graph here otherwise Pytorch will throw it away)
loss.backward(retain_graph=True)

print(w.grad.data)

# manually zero out the gradient
w.grad.zero_()
pass

Does this gradient match with the one you compute manually?

With the gradients, we can update the weights and biases using gradient descent:
$$w_{k+1}=w_{k} - \alpha\frac{\partial L}{\partial w_k}$$
where $w$ is the parameter we are updating, $\alpha$ is the learning rate, and $\frac{\partial L}{\partial w_k}$ is the gradient at step $k$. You can learn more about gradient descent [here](https://en.wikipedia.org/wiki/Gradient_descent) and [here](https://developers.google.com/machine-learning/crash-course/reducing-loss/gradient-descent)

**TODO 8:** Implement a single step of gradient descent in function `gradient_descent_step` in `proj5_code/classification/optimizer.py`

In [ ]:
print("Testing gradient descent step: ", verify(test_gradient_descent_step))

Let's take one step of the gradient descent and check if the the loss value decreased

In [ ]:
# set learning rate
lr = .03

loss = compute_quadratic_loss(w)
gradient_descent_step(w, loss, lr) 

In [ ]:
loss = compute_quadratic_loss(w)
print('w={:.4f}\tLoss={:.4f}'.format(w.detach().numpy().item(), loss.detach().numpy().item()))

Looks like it's been optimized!

Now let's run a few more updates and see where we can get!

In [ ]:
import time


start = time.time()
for i in range(200):
    loss = compute_quadratic_loss(w)
    if not (i+1)%10:
        print('Iteration {}: w={:.4f}\tLoss={:.4f}'.format(
            i+1, w.detach().numpy().item(), loss.detach().numpy().item()))
        
    gradient_descent_step(w, loss, lr) 
        
print('\noptimization takes %0.3f seconds'%(time.time()-start))

Seems that it's doing a great job training our model! The loss now has decreased significantly to a pretty small value.

### 4.2 Optimization using Pytorch's gradient descent optimizer

Now let's see how we can simplify this using the `torch.optim` package from PyTorch. You can see that using optimizer from `torch.optim` package can achieve the same results with a lot less code from our side. Also, there are many features available over the vanilla gradient descent. Let's use the Stochastic Gradient Descent (SGD) optimizer available in Pytorch.

In [ ]:
from torch.optim import SGD

# define parameters we want to optimize
w = torch.tensor([0.0], requires_grad=True)

optimizer = SGD([w], lr=lr)

start = time.time()
for i in range(200):
    loss = compute_quadratic_loss(w)   
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if not (i+1)%10:
        print('Iteration {}: w={:.4f}\tLoss={:.4f}'.format(
            i+1, w.detach().numpy().item(), loss.detach().numpy().item()))
        
print('\noptimization takes %0.3f seconds'%(time.time()-start))

### 4.3 Setting up the optimizer for SimpleNet

**TODO 9:** **initialize the following cell with proper values for learning rate and weight decay** 

**Note:** There is nothing to do in this TODO for the first pass. You'll train the model with these values and it will be bad. Then you can come back here and tune the parameters.

In [ ]:
# TODO: add a decent initial setting and tune from there
optimizer_config = {
  "optimizer_type": "adam",
  "lr": 1,
  "weight_decay": 1e-2
}

We will now set up a utility function to define an optimizer on the loss for a model.

**TODO 10:** complete the ```get_optimizer()``` function in ```proj5_code/classification/optimizer.py```. The helper function accepts three basic configurations as defined below. Any other configuration is optional. *SGD* optimizer type should be supported, anything else is optional.

In [ ]:
optimizer = get_optimizer(simple_model, optimizer_config)

## 5 Training SimpleNet

We have completed all the components required to train the first model for this course. Let's pass in the model architecture, optimizer, transforms for both the training and testing datasets into the trainer, and proceed to the next cell to train it. If you have implemented everything correctly, you should be seeing a decreasing loss value.

**Note** in this project, we will be using the test set as the validation set (i.e. using it to guide our decisions about models and hyperparamters while training). In actual practise, you would not interact with the test set until reporting the final results.

**Note** that your CPU should be sufficient to handle the training process for all networks in this project, and the following training cells will take less than 5 minutes; you may also want to decrease the value for `num_epochs` and quickly experiment with your parameters. The default value of **30** is good enough to get you around the threshold for Part 1, and you are free to increase it a bit and adjust other parameters in this part.

In [ ]:
# re-init the model so that the weights are all random
simple_model = SimpleNet()
optimizer = get_optimizer(simple_model, optimizer_config)

trainer = Trainer(data_dir=data_base_path, 
                  model = simple_model,
                  optimizer = optimizer,
                  model_dir = os.path.join(model_base_path, 'simple_net'),
                  train_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  test_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  batch_size = 32,
                  load_from_disk = False,
                  cuda = is_cuda
                 )

In [ ]:
import time
simple_net_start = time.time()
trainer.train(num_epochs=30)
simple_net_end = time.time()
print("The training time taken for simple net is {:.9f}".format(simple_net_end-simple_net_start))

Now let's have your model predict on some examples and see how well it performs qualitatively. You should try the following cell multiple times to understand whats happening.

In [ ]:
# visualize train split
print("Examples from train split:")
visualize(simple_model, 'train', get_fundamental_transforms(inp_size, dataset_mean, dataset_std), data_base_path)

In [ ]:
# visualize test split
print("Examples from test split:")
visualize(simple_model, 'test', get_fundamental_transforms(inp_size, dataset_mean, dataset_std), data_base_path)

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

In [ ]:
train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print('Train Accuracy = {:.4f}; Validation Accuracy = {:.4f}'.format(train_accuracy, validation_accuracy))

In [ ]:
print('Testing simple net weights saved: ', verify(test_simple_net_checkpoint))

After you have finished the training process, now plot out the loss and accuracy history. You can also check out the final accuracy for both training and validation data. Copy the accuracy plots and values onto the report, and answer the questions there. 

**TODO 11:** Obtain a **45%** validation accuracy to receive full credits for Part 1. You can go back to TODO 8 first to tune your paramters for optimization using the following tips:

**Tips**:
1. If the loss decreases very slowly, try increasing the value of the lr (learning rate).
2. Initially keep the value of weight decay (L2-regularization) very low. Even zero is fine. Use the weight decay as the last resort.
3. Try to first adjust lr in multiples of 3 initially. When you are close to reasonable performance, do a more granular adjustment. Do try to keep lr values less than 0.1.
4. If you want to increase the validation accuracy by a little bit, try increasing the weight_decay to prevent overfitting. Do not use tricks from Section 6 just yet.

If you still need to tweak the model architecture, you are free to do so. But remember complex models will require more time to train, and TAs could achieve ~50% accuracy with the described model.

## EC1 Solving overfitting
We have obtained a 45% accuracy on the validation data with a simple model; Feeling even better for the training accuracy right? More than 90% (if you have implemented everything correctly). But should you?

Our final accuracies for training and validation data differ a lot from each other, which indicates that the model we have defined **fits too well with the training data, but is unable to generalize well on data it has not trained on**: this is often regarded as **overfitting**. In this section we are going to apply 2 techniques to tackle with it: adjusting both data and model.

References:
- https://www.cs.princeton.edu/courses/archive/spring16/cos495/slides/ML_basics_lecture6_overfitting.pdf

### EC1.1 Jitter, Random Flip, and Normalization
One common technique to increase the "variability" of the data is to **augment** it. Firstly, we don't have a huge amount of data, so let's "jitter" it; secondly, when you mirror an image of a **kitchen**, you can tell that the mirrored image is still a kitchen. 

**TODO 12:** finish the `get_data_augmentation_transforms()` function in `proj5_code/classification/data_transforms.py`: you should first copy your existing fundamental transform implementation into this function, and then insert other which help you do the above adjustment before the fundamental ones.

Tips: Try jittering the colors, and flipping the image horizontaly. These two operations wont change the scene contents. You can also try resizing and cropping. 

You are free to experiment with different kinds of augmentation techniques by adding new techniques or replacing existing techniques

In [ ]:
inp_size = (64,64)

### EC1.2 Dropout
"Dropout" is a technique commonly used to regularize the network. It randomly turns off the connection between neurons inside the network and prevent the network from relying too much on a specific neuron. 

References: 
- https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf
- https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/

**TODO 13:** finish the `proj5_code/classification/simple_net_dropout.py` with your previous SimpleNet model, plus the dropout layer. Think where you want the dropout should be places: fully connected layers, conv layers, or both? Generally the layer with more number of parameters might be more prone to overfitting.

If you are skipping this part, copy over the exact simple net arhcitecute. You will fail the following unit test but you can ignore it.

In [ ]:
print("Testing your SimpleNetDropout architecture: ", verify(test_simple_net_dropout))

In [ ]:
simple_model_dropout = SimpleNetDropout()
print(simple_model_dropout)

Similar to the previous part, **initialize the following cell with proper values for learning rate and weight decay**.

In [ ]:
# TODO: add a decent initial setting and tune from there
optimizer_config = {
  "optimizer_type": "adam",
  "lr": 1,
  "weight_decay": 1e-2
}

In [ ]:
simple_model_dropout = SimpleNetDropout()
optimizer = get_optimizer(simple_model_dropout, optimizer_config)

trainer = Trainer(data_dir=data_base_path, 
                  model = simple_model_dropout,
                  optimizer = optimizer,
                  model_dir = os.path.join(model_base_path, 'simple_net_dropout'),
                  train_data_transforms = get_data_augmentation_transforms(inp_size, dataset_mean, dataset_std),
                  test_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  batch_size = 32,
                  load_from_disk = False,
                  cuda = is_cuda
                 )

The following cell will take longer than Part 1, as now we have more data (and more variability), and the model is slightly more complicated than before as well; however, it should finish within 10~15 minutes anyway, and the default `num_epochs` is also good enough as a starting point for you to pass this part.

In [ ]:
trainer.train(num_epochs=30)

Now let's have your model predict on some examples and see how well it performs qualitatively.

In [ ]:
# # visualize train split
print("Examples from train split:")
visualize(simple_model_dropout, 'train', get_fundamental_transforms(inp_size, dataset_mean, dataset_std))

In [ ]:
# # visualize test split
print("Examples from test split:")
visualize(simple_model_dropout, 'test', get_fundamental_transforms(inp_size, dataset_mean, dataset_std))

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

In [ ]:
train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print('Train Accuracy = {:.4f}; Validation Accuracy = {:.4f}'.format(train_accuracy, validation_accuracy))

Similar to the previous part, now plot out the loss and accuracy history. Also copy the plots onto the report, and answer the questions accordingly.

**TODO 14:** Achieve **52%** validation accuracy for full credits for this part.

## 6 Semantic Segmentation with Deep Learning

### 6.1: A Simple Segmentation Baseline
We'll start with a very simple baseline -- a pretrained ResNet-50, without the final averagepool/fc layer, and a single 1x1 conv as a final classifier, converting the (2048,7,7) feature map to scores over 11 classes, a (11,7,7) tensor. Note that our output is just 7x7, which is very low resolution. 

**TODO 15:** Implement upsampling to the original height and width, and compute the loss and predicted class per pixel in `proj_5_code/segmentation/simple_segmentation_net.py`. The loss function has already been set in the SimpleSegmentationNet class (`self.criterion`).

In [ ]:
from proj5_code.proj5_unit_tests.segmentation.test_simple_segmentation_net import (
    test_check_output_shapes,
    test_check_output_shapes_testtime,
)
test_check_output_shapes()

print("test_check_output_shapes(): ", verify(test_check_output_shapes))
print("test_check_output_shapes_testtime(): ", verify(test_check_output_shapes_testtime))

**TODO 16:** Now that you've completed your implementtion of SimpleSegmentationNet, train the network using the `proj5_code/proj5_colab.ipynb` notebook. Achieve at least 40% mIoU in order to achieve full credit.

## EC 2: PSPNet and ResNet-50


### PSPNet and ResNet-50

We'll be implementing PSPNet for this project, which uses a ResNet-50 backbone. First, we will provide some background information to familiarize yourself with the network and architecture.

ResNet-50 has 50 convolutional layers, which is significantly deeper than your SimpleNet of Project 5. We give you the implementation in `proj5_code/segmentation/resnet.py`. 

The ResNet-50 is composed of 4 different sections (each called a "layer"), named `layer1`, `layer2`, `layer3`, `layer4`. Each layer is composed of a repeated number of blocks, and each such block is named a `BottleNeck`. Specifically, `layer1` has 3 Bottlenecks, `layer2` has 4 Bottlenecks, `layer3` has 6 Bottlenecks, and `layer4` has 3 Bottlenecks. In all, ResNet-50 has 16 Bottlenecks, which accounts for 48 of the conv layers.

### Visualizing a ResNet Bottleneck Module

The BottleNeck has a residual connection, from which ResNet gets its name:

<img width="300" src="https://user-images.githubusercontent.com/16724970/114430171-2ac1c200-9b8c-11eb-8341-fc943ff0945f.png">

See Figure 5 of the [ResNet paper](https://arxiv.org/pdf/1512.03385.pdf)

### Implementing a Bottleneck

The Bottleneck is implemented exactly as the figure above shows, with 1x1 Conv -> BN -> ReLU -> 3x3 Conv -> BN -> ReLU -> 1x1 Conv -> BN -> Optional Downsample -> Add Back Input -> ReLU. The channel dimension of the feature map will be expanded by 4x, as we can see by the conv layer `in_features` and `out_features` parameters. And notice that the stride is set at the `conv2` module, which will be very important later.

```python
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
```

and the forward method of the `Bottleneck` shows the residual connection. Notice that when we add back the input (the identity operation), we may need to downsample it for the shapes to match during the add operation (if the main branch downsampled the input):
```python
    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
```

### Visualizing the Architecture
Plotting the whole network architecture would require a massive figure, but we can show how data flows through just one Bottleneck, starting with 64 channels, and ending up with 256 output channels:
<p float="left">
  <img src="https://user-images.githubusercontent.com/16724970/114427960-9eae9b00-9b89-11eb-9a3b-96817f205f32.png" width="400" />
</p>


**TODO 17:** The first part of the PSPNet is to initialize the Resnet-50 backbone in `proj5_code/segmentation/pspnet.py`, specifically in the `__init__` function.

### Background: Pyramid Pooling Module (PPM)
One major component of the PSPNet is the PPM Module. After feeding an image through the ResNet backbone and obtaining a feature map, PSPNet aggregates context over different portions of the image with the PPM.

The PPM splits the $H \times W$ feature map into KxK grids. Here, 1x1, 2x2, 3x3,and 6x6 grids are formed, and features are average-pooled within each grid cell. Afterwards, the 1x1, 2x2, 3x3, and 6x6 grids are upsampled back to the original $H \times W$ feature map resolution, and are stacked together along the channel dimension. These grids are visualized below (center):

<img src="https://user-images.githubusercontent.com/16724970/114436422-4b414a80-9b93-11eb-8f02-8e7506b5f9a1.jpg" width="900">

We have ALREADY implemented this for you in `proj5_code/segmentation/ppm.py`.

**TODO 18:** Initialize the PPM Module in `proj5_code/segmentation/pspnet.py`. 

### Net Surgery for Increased Output Resolution and Receptive Field
The basic ResNet-50 has two major problems:
1. It does not have a large enough receptive field
2. If run fully-convolutionally, it produces a low-resolution output (just $7 \times 7$)!

To fix the first problem, will need to replace some of its convolutional layers with dilated convolution. To fix the second problem, we'll reduce the stride of the network from 2 to 1, so that we don't downsample so much. Instead of going down to 7x7, we'll reduce to 28x28 for 224x224 input, or 26x26 for 201x201, like we do in this project. In other words, the downsampling rate will go from (1/32) to just (1/8).

These animations depict how the dilated convolution (i.e. with dilation > 1) operation compares to convolution with no dilation (i.e. with dilation=1).

Conv w/ Stride=1, Dilation=1 | Conv w/ Stride=2, Dilation=1 | Conv w/ Stride=1, Dilation=2
:-: | :-: | :-:
<img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif" width="300" align="center"> | <img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif" width="300" align="center"> | <img src="https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/dilation.gif" width="300" align="center"> 


In Layer3, in every `Bottleneck`, we will change the 3x3 `conv2`, we will replace the conv layer that had stride=2, dilation=1, and padding=1 with a new conv layer, that instead  has stride=1, dilation=2, and padding=2. In the `downsample` block, we'll also need to hardcode the stride to 1, instead of 2.

In Layer4, for every `Bottleneck`, we will make the same changes, except we'll change the dilation to 4 and padding to 4.


**TODO 19:** Complete the rest of the function in `proj5_code/segmentation/pspnet.py`:

- `__replace_conv_with_dilated_conv`
- `__create_classifier`
- `forward`


In [ ]:
from proj5_code.proj5_unit_tests.segmentation.test_pspnet import (
    test_pspnet_output_shapes,
    test_check_output_shapes_testtime_pspnet,
    test_pspnet_output_with_zoom_factor
)

print("test_pspnet_output_shapes():", verify(test_pspnet_output_shapes))
print("test_check_output_shapes_testtime_pspnet(): ", verify(test_check_output_shapes_testtime_pspnet))
print("test_pspnet_output_with_zoom_factor(): ", verify(test_pspnet_output_with_zoom_factor))

**TODO 20:** Now that you've completed your implementtion of PSPNet, train the network using the `proj5_code/proj5_colab.ipynb` notebook. Achieve at least 60% mIoU in order to achieve full extra credit.